In [1]:
import os
import subprocess
import time
import sys
import threading
import socket
from urllib.request import Request, urlopen
import re
# Function to install packages
def install_packages():
    subprocess.check_call([sys.executable, "-m", "pip", "install", "streamlit", "pyngrok", "pyjwt", "watchdog"])
print("Installing required packages...")
install_packages()
# Import after installation
from pyngrok import ngrok
# --- Create Streamlit Config for Dark Theme ---
os.makedirs(".streamlit", exist_ok=True)
config_toml = """
[theme]
base="dark"
primaryColor="#4F8BF9"
backgroundColor="#0E1117"
secondaryBackgroundColor="#262730"
textColor="#FAFAFA"
font="sans serif"
[server]
headless = true
"""
with open(".streamlit/config.toml", "w") as f:
    f.write(config_toml)
print("Applied Dark Theme configuration.")

Installing required packages...
Applied Dark Theme configuration.


In [2]:
# --- Create the Streamlit App File ---
app_code = """
import streamlit as st
import jwt
import datetime
import time
import re

# --- Configuration ---
SECRET_KEY = "super_secret_key_for_demo"
ALGORITHM = "HS256"
ACCESS_TOKEN_EXPIRE_MINUTES = 30

# --- JWT Utils ---
def create_access_token(data: dict):
    to_encode = data.copy()
    expire = datetime.datetime.utcnow() + datetime.timedelta(minutes=ACCESS_TOKEN_EXPIRE_MINUTES)
    to_encode.update({"exp": expire})
    encoded_jwt = jwt.encode(to_encode, SECRET_KEY, algorithm=ALGORITHM)
    return encoded_jwt

def verify_token(token: str):
    try:
        payload = jwt.decode(token, SECRET_KEY, algorithms=[ALGORITHM])
        return payload
    except:
        return None

# --- Validation Helpers ---
def is_empty(value):
    return value is None or value.strip() == ""

def is_valid_email(email):
    if is_empty(email):
        return False
    pattern = r"^[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}$"
    return bool(re.match(pattern, email))

def is_valid_password(password):
    if is_empty(password):
        return False
    if len(password.strip()) < 8:
        return False
    if not password.isalnum():
        return False
    return True

# --- Session Initialization ---
if "jwt_token" not in st.session_state:
    st.session_state.jwt_token = None

if "page" not in st.session_state:
    st.session_state.page = "login"

if "users" not in st.session_state:
    st.session_state.users = {}

# --- Security Questions ---
SECURITY_QUESTIONS = [
    "What is your pet name?",
    "What is your mother’s maiden name?",
    "What is your favorite teacher?"
]

# ============================
# LOGIN PAGE
# ============================
def login_page():
    st.title("Login")

    with st.form("login_form"):
        email = st.text_input("Email")
        password = st.text_input("Password", type="password")
        submit = st.form_submit_button("Login")

        if submit:
            if is_empty(email) or is_empty(password):
                st.error("Email and Password cannot be empty or spaces.")
                return

            user = st.session_state.users.get(email.strip())

            if user and user["password"] == password.strip():
                token = create_access_token({"sub": email, "username": user["username"]})
                st.session_state.jwt_token = token
                st.success("Login successful!")
                time.sleep(0.5)
                st.rerun()
            else:
                st.error("Invalid email or password")

    col1, col2 = st.columns(2)
    with col1:
        if st.button("Create Account"):
            st.session_state.page = "signup"
            st.rerun()

    with col2:
        if st.button("Forgot Password"):
            st.session_state.page = "forgot_password"
            st.rerun()

# ============================
# SIGNUP PAGE
# ============================
def signup_page():
    st.title("Create Account")

    with st.form("signup_form"):
        username = st.text_input("Username")
        email = st.text_input("Email")
        password = st.text_input("Password", type="password")
        confirm_password = st.text_input("Confirm Password", type="password")
        security_question = st.selectbox("Security Question", SECURITY_QUESTIONS)
        security_answer = st.text_input("Security Answer")

        submit = st.form_submit_button("Sign Up")

        if submit:
            errors = []

            if is_empty(username):
                errors.append("Username cannot be empty")

            if not is_valid_email(email):
                errors.append("Invalid email")

            if email.strip() in st.session_state.users:
                errors.append("Email already exists")

            if not is_valid_password(password):
                errors.append("Password must be 8+ alphanumeric characters")

            if password != confirm_password:
                errors.append("Passwords do not match")

            if is_empty(security_answer):
                errors.append("Security answer cannot be empty or spaces")

            if errors:
                for e in errors:
                    st.error(e)
            else:
                st.session_state.users[email.strip()] = {
                    "username": username.strip(),
                    "password": password.strip(),
                    "security_question": security_question,
                    "security_answer": security_answer.strip().lower()
                }

                st.success("Account created successfully!")
                st.session_state.page = "login"
                st.rerun()

    if st.button("Back to Login"):
        st.session_state.page = "login"
        st.rerun()

# ============================
# FORGOT PASSWORD FLOW
# ============================
def forgot_password_page():
    st.title("Forgot Password")

    email = st.text_input("Enter your registered Email")

    if st.button("Verify Email"):
        if is_empty(email):
            st.error("Email cannot be empty")
            return

        user = st.session_state.users.get(email.strip())

        if user:
            st.session_state.reset_email = email.strip()
            st.success("Email found!")
        else:
            st.error("Email not registered")

    if "reset_email" in st.session_state:
        user = st.session_state.users.get(st.session_state.reset_email)

        st.write("Security Question:")
        st.info(user["security_question"])

        answer = st.text_input("Enter Security Answer")

        if st.button("Verify Answer"):
            if is_empty(answer):
                st.error("Security answer cannot be empty")
                return

            if answer.strip().lower() == user["security_answer"]:
                st.session_state.allow_password_reset = True
                st.success("Answer verified! Set new password.")
            else:
                st.error("Incorrect security answer")

    if st.session_state.get("allow_password_reset"):
        new_password = st.text_input("New Password", type="password")

        if st.button("Update Password"):
            if not is_valid_password(new_password):
                st.error("Password must be 8+ alphanumeric characters")
                return

            st.session_state.users[st.session_state.reset_email]["password"] = new_password.strip()
            st.success("Password updated successfully!")

            del st.session_state["reset_email"]
            del st.session_state["allow_password_reset"]

            st.session_state.page = "login"
            time.sleep(1)
            st.rerun()

    if st.button("Back to Login"):
        st.session_state.page = "login"
        st.rerun()

# ============================
# DASHBOARD
# ============================
def dashboard_page():
    payload = verify_token(st.session_state.jwt_token)

    if not payload:
        st.session_state.jwt_token = None
        st.session_state.page = "login"
        st.rerun()

    st.sidebar.title("Menu")
    if st.sidebar.button("Logout"):
        st.session_state.jwt_token = None
        st.session_state.page = "login"
        st.rerun()

    st.title(f"Welcome {payload['username']} 👋")
    st.write("This is your dashboard.")

# ============================
# MAIN ROUTER
# ============================
if st.session_state.jwt_token:
    dashboard_page()
else:
    if st.session_state.page == "signup":
        signup_page()
    elif st.session_state.page == "forgot_password":
        forgot_password_page()
    else:
        login_page()
"""
with open("app.py", "w") as f:
    f.write(app_code)
print("Streamlit app code written to 'app.py'")

Streamlit app code written to 'app.py'


<>:36: SyntaxWarning: invalid escape sequence '\.'
<>:36: SyntaxWarning: invalid escape sequence '\.'
/tmp/ipython-input-4187962931.py:36: SyntaxWarning: invalid escape sequence '\.'
  pattern = r"^[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}$"


In [ ]:
# --- Wait for Streamlit to Start ---
def wait_for_streamlit(port=8501, timeout=30):
    start_time = time.time()
    while time.time() - start_time < timeout:
        try:
            sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            sock.settimeout(1)
            result = sock.connect_ex(('localhost', port))
            if result == 0:
                sock.close()
                return True
            sock.close()
        except Exception:
            pass
        time.sleep(1)
    return False
# --- Ngrok Setup ---
print("\nTo access the app, you need an Ngrok Authtoken.")
print("Get it from: https://dashboard.ngrok.com/get-started/your-authtoken")
authtoken = input("Enter your Ngrok Authtoken: ").strip()
if authtoken:
    ngrok.set_auth_token(authtoken)

    # Kill any existing ngrok process
    os.system("pkill ngrok")
    os.system("pkill streamlit")

    # Run Streamlit in the background FIRST
    print("Starting Streamlit...")
    # Using Subprocess.Popen to run in background
    # Redirecting output to /dev/null to keep cell clean
    process = subprocess.Popen(["streamlit", "run", "app.py", "--server.port", "8501", "--server.address", "localhost"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

    # Wait for it to be ready
    if wait_for_streamlit():
        print("Streamlit is active! Connecting Ngrok...")
        # Open a tunnel to the streamlit port 8501
        try:
            public_url = ngrok.connect(8501).public_url
            print(f"\n🚀 Streamlit App is running!")
            print(f"👉 Public URL: {public_url}")
            print("\n(Click the URL above to open the app)")

            # Keep main thread alive
            try:
                # Keep checking if process is alive
                while process.poll() is None:
                    time.sleep(1)
            except KeyboardInterrupt:
                print("Stopping...")
                ngrok.disconnect(public_url)
                process.terminate()
        except Exception as e:
            print(f"Ngrok connection failed: {e}")
            process.terminate()
    else:
        print("Error: Streamlit failed to start in time.")
        process.terminate()
else:
    print("Ngrok Authtoken is required to expose the app publicly.")


To access the app, you need an Ngrok Authtoken.
Get it from: https://dashboard.ngrok.com/get-started/your-authtoken
Enter your Ngrok Authtoken: 39YluLaya60LRlucUdQkCf5lEQD_5ZsU4ZdbpkBxxrFxPqERX
Starting Streamlit...
Streamlit is active! Connecting Ngrok...

🚀 Streamlit App is running!
👉 Public URL: https://unportionable-nonsacrilegiously-shani.ngrok-free.dev

(Click the URL above to open the app)
